# FHIR-Drills (Simple Search)

This notebook is a rough, Jupyter translation of the FHIR-Drills tutorial based on Postman avaialble [here](https://fhir-drills.github.io/simple-patient.html). Rather than using the Python Smart on FHIR library, I'm going to use the Python requests library so that we are working more directly with the HTTP protocol and are closer to the steps described in the original tutorial than would be achieved with the FHIR Python client.

For more details, please refer to the original tutorials.

FHIR is built upon web standards, particularly [Representational state transfer (REST)](https://en.wikipedia.org/wiki/Representational_state_transfer). 

In this tutorial we will explore RESTful operations to create, read, update, and delete resources (CRUD) with a FHIR server. 

We will interact with the FHIR server using the Hype Text Transfer Protocol [(HTTP)](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol), a digital communications protocol develooped by Tim Berners-Lee at CERN. We will use the Python package [`requests`](https://requests.readthedocs.io/en/master/index.html) to implement the HTTP commands for us. `requests` will generate URLs that tell the remote server what we want to do, receive the responses from the remote servers, and convert them into Python objects we can interact with.

## Find a patient with relevant variables

In [1]:
import requests
from post_data import create_resource
import pprint
pp = pprint.PrettyPrinter(indent=2)

### Create Patient and get `id`

I've written some Python code to upload a patient to the server behidn the scences, mimicing what the HTML button does on the original tutorial

When we invoke `create_resource()` we upload a JSON file describing a patient. When this is posted (inserted) into the remote server, this patient is assigned a unique identifier which is necessary to interact with the patient data remotely. This identifer is then stored in the variable `patientId`.

In [2]:
patientId = create_resource()

In [3]:
patientId

'0c215ce8-2dc0-41d6-b750-2a7eddc7f68d'

### Set the headers to work with FHIR data

Data transfer with HTTP relies on specifying the [media type (MIME type)](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol). This is done with the HTTP header. So we need to specify that we will be using FHIR data. We will also specify what character encoding we will be using.

Finally, we need to know the Uniform Resource Locator (URL) of the FHIR server.

In [4]:
FHIRJSONMimeType = 'application/fhir+json'
header_defaults = {
            'Accept': FHIRJSONMimeType,
            'Accept-Charset': 'UTF-8',
        }
URL = 'https://stu3.test.pyrohealth.net/fhir'

To request the FHIR resource, we build up a URL based on

- The base URL: `URL`
- The resource type we want: e.g. `Patient`
- The ID of the resoruce we want: e.g. `patientID` (returned from `create_resource()`)

In [5]:
rurl = URL+ '/Patient/'+patientId
rurl

'https://stu3.test.pyrohealth.net/fhir/Patient/0c215ce8-2dc0-41d6-b750-2a7eddc7f68d'

## Get Our Patient

We will use the HTTP [GET](https://www.w3.org/Protocols/rfc2616/rfc2616-sec9.html) method.

We will construct a query by passing values using the [params](https://requests.readthedocs.io/en/master/user/quickstart/#passing-parameters-in-urls) keyword argument.

In [6]:
r = requests.get(URL+ '/Patient/',
                 headers = header_defaults, params={"family":"fhirman"})

`r` contains the results of our request. It has multiple parts

- `r.status_code`: Is the code returned by the remote server. `200` means success! I ran across many bad codes trying to get things working.
- `r.headers`: The metadata about the request and response
- `r.text`:
- `r.json`: A method to convert the data returned by the server into a Python dictionary.

In [7]:
r.request.body

In [8]:
print(r.status_code)
print(r.headers)

200
{'Content-Type': 'application/fhir+json; charset=utf-8', 'Content-Encoding': 'gzip', 'Server': 'Microsoft-IIS/8.5', 'X-Powered-By': 'ASP.NET', 'Date': 'Mon, 12 Jul 2021 23:39:57 GMT', 'Content-Length': '5547'}


In [9]:
print(r.text)

In [10]:
pp.pprint(r.json())

{ 'entry': [ { 'fullUrl': 'https://stu3.test.pyrohealth.net/fhir/Patient/9db35bee-3997-49f6-a59d-9d38df6a5c06',
               'resource': { 'address': [ { 'city': 'Brisbane',
                                            'country': 'AUS',
                                            'line': ['400 George Street'],
                                            'postalCode': '4000',
                                            'state': 'QLD',
                                            'use': 'work'}],
                             'birthDate': '1973-09-30',
                             'gender': 'male',
                             'id': '9db35bee-3997-49f6-a59d-9d38df6a5c06',
                             'identifier': [ { 'system': 'http://ns.electronichealth.net.au/id/hi/ihi/1.0',
                                               'type': { 'coding': [ { 'code': 'NI',
                                                                       'display': 'National '
                                   

'period': { 'start': '2001-05-06'},
                                               'system': 'urn:oid:1.2.36.146.595.217.0.1',
                                               'type': { 'coding': [ { 'code': 'MR',
                                                                       'system': 'http://hl7.org/fhir/v2/0203'}]},
                                               'use': 'usual',
                                               'value': '6666'}],
                             'maritalStatus': { 'coding': [ { 'code': 'M',
                                                              'display': 'Married',
                                                              'system': 'http://hl7.org/fhir/v3/MaritalStatus'}]},
                             'meta': { 'lastUpdated': '2019-01-30T11:57:55.305+00:00',
                                       'versionId': '2'},
                             'name': [ { 'family': 'Fhirman',
                                         'given': ['Steve'],
  

## Search Options

FHIR defines eight different search datatypes as follows: 
                        
<li><a href="http://hl7.org/fhir/STU3/search.html#number">Number</a></li>
                        <li><a href="http://hl7.org/fhir/STU3/search.html#date">Date/DateTime</a></li>
                        <li><a href="http://hl7.org/fhir/STU3/search.html#string">String</a></li>
                        <li><a href="http://hl7.org/fhir/STU3/search.html#token">Token</a></li>
                        <li><a href="http://hl7.org/fhir/STU3/search.html#reference">Reference</a></li>                        
                        <li><a href="http://hl7.org/fhir/STU3/search.html#composite">Composite</a></li>
                        <li><a href="http://hl7.org/fhir/STU3/search.html#quantity">Quantity</a></li>
                        <li><a href="http://hl7.org/fhir/STU3/search.html#uri">URI</a></li>

- [FHIR version 3 Search API](http://hl7.org/fhir/STU3/search.html#number)
- [FHIR version 4 Search API](http://hl7.org/fhir/search.html)

## Questions

1. What datatype does the search parameter length use on the Encounter resource?
1. Using the references above try and find the patient with an old address in the suburb of 'Erinsborough'?
1. Try and find the patient name for the person with an IHI (Individual Health Identifier) of __'8003601043886154'__?

## Question 2

The description of addresses is provided [here](http://hl7.org/fhir/STU3/datatypes.html#address).

I can't currently remember where I found the `address-city` and `address-use` as the semantics to reference the components of an address.

In [0]:
r2 = requests.get(URL+ '/Patient/',
                 headers = header_defaults, params={"address-city":"Erinsborough",
                                                    "address-use":"old"})

In [0]:
r2.status_code

In [0]:
pp.pprint(r2.json())

## Question 3

In [0]:
r3 = requests.get(URL+ '/Patient/',
                 headers = header_defaults, params={"identifier":"IHI", 
                                                    "identifier":"8003601043886154"})

In [0]:
r3rslts = r3.json()

In [0]:
r3entries = r3rslts['entry']
len(r3entries)

In [0]:
for rt in r3entries:
    pp.pprint(rt)
    print('-'*47)

## Question

If you repeat the `simple patient` tutorial but do not delete your patient, can you create a search to find yourself (that is the patient with Sam Fhirman's name replaced with your own)?